## <font color=yellow>Example</font>
- 이 모든 것을 어떻게 결합할 수 있는지 살펴보겠음
    - 아래는 생성한 Assistant를 사용하기 위해 필요한 모든 코드

- 우리가 이미 수학 Assistant를 만들었기 때문에, 그 ID를 `MATH_ASSISTANT_ID`에 저장했음 
- 그런 다음 두 개의 함수를 정의했음

    - `submit_message`: Thread에 Message를 생성한 다음, 새로운 Run을 시작하고 반환
    - `get_response`: Thread의 메시지 목록을 반환

In [23]:
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [24]:
from openai import OpenAI

client = OpenAI()
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘",
    model="gpt-4o",
)

# 여기서 assistant를 만들어도 되고, OpenAI 홈페이지에서 만들어서 지정해줘도 됨
MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

또한 재사용할 수 있는 `create_thread_and_run` 함수를 정의했는데, 이는 실제로 우리 API의 [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) 복합 함수와 거의 동일함 
- 마지막으로, 우리는 각각의 가상 사용자 요청을 새로운 Thread에 제출할 수 있음

이 모든 API 호출은 비동기 작업이라는 점에 유의
- 이는 우리가 `asyncio`와 같은 비동기 라이브러리의 사용 없이도 실제로 코드에서 비동기 동작을 얻을 수 있음을 의미

In [25]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run

# Emulating concurrent user requests
thread1, run1 = create_thread_and_run("방정식 '3x + 11 = 14'를 풀어줘")
thread2, run2 = create_thread_and_run("선형대수에 대해 설명해줘")
thread3, run3 = create_thread_and_run("나는 수학을 싫어해. 어떻게 하면 좋을까?")

# Now all Runs are executing...

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\1054163418.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:14: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:17: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.create(


모든 Run이 진행되고 나면, 각각을 기다린 후 응답을 받을 수 있음

In [26]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "고마워")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\2754427285.py:14: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:23: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.messages.list(thread_id=thread.id, order="asc")


# Messages
user: 방정식 '3x + 11 = 14'를 풀어줘
assistant: \( x = 1 \)

# Messages
user: 선형대수에 대해 설명해줘
assistant: 선형대수는 벡터와 행렬을 중심으로 공부하는 수학의 한 분야입니다.

# Messages
user: 나는 수학을 싫어해. 어떻게 하면 좋을까?
assistant: 작은 목표부터 시작해 재미있는 방법으로 접근해 보세요.



C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:17: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.create(


# Messages
user: 나는 수학을 싫어해. 어떻게 하면 좋을까?
assistant: 작은 목표부터 시작해 재미있는 방법으로 접근해 보세요.
user: 고마워
assistant: 천만에요! 도움이 필요하면 언제든지 말씀하세요.



위 assistant는 수학에 특화되어 있지 않음
- 수학에 특화되게 만드려면, Code Interpreter라는 것을 추가할 수 있음
- Code Interpreter: 코드를 생성하고, 코드를 실행할 수 있는 능력 (코드를 통해 연산을 함)

## <font color=yellow>Tools</font>

- Assistants API의 핵심 기능 중 하나는 Code Interpreter, Retrieval 및 사용자 정의 Functions과 같은 도구를 우리의 Assistants에 장착할 수 있는 능력 

### Code Interpreter

- 우리의 수학 과외 선생님에게 [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter) 도구를 장착해보겠음 
- 이 작업은 대시보드에서 수행할 수 있음


In [27]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [28]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

{'id': 'asst_J8WwgAVAMAsKm3t5uPDLX5gA',
 'created_at': 1753188046,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': []}, 'file_search': None},
 'top_p': 1.0,
 'reasoning_effort': None}

In [29]:
thread, run = create_thread_and_run("코드를 사용하여 처음 10개의 피보나치 숫자를 생성해줘")

run = wait_on_run(run, thread)
pretty_print(get_response(thread))

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\1054163418.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:14: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:17: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.create(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:23: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.messages.list(thread_id=thread.id, order="asc")


# Messages
user: 코드를 사용하여 처음 10개의 피보나치 숫자를 생성해줘
assistant: 처음 10개의 피보나치 숫자는 [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]입니다.



일부 사용 사례에서는 이 정도면 충분할 수 있음 
- 하지만 Assistant가 정확히 무엇을 하는지 더 자세히 알고 싶다면 Run의 Steps를 살펴볼 수 있음

## <font color=yellow>Steps</font>

Run은 하나 이상의 Steps로 구성됨 
- Run과 마찬가지로 각 Step에는 조회할 수 있는 `status`가 있음 
- 이는 사용자에게 Step의 진행 상황을 표시하는 데 유용함
    - 예: Assistant가 코드를 작성하거나 검색을 수행하는 동안 스피너 표시

In [30]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, 
    run_id=run.id, 
    order="asc",
)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\1035908624.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


각 Step의 `step_details`를 살펴보겠음

In [31]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_c2kA9fswBXFNCRbDxuutDETw',
   'code_interpreter': {'input': '# Generate the first 10 Fibonacci numbers using a loop\n\ndef fibonacci_sequence(n):\n    sequence = [0, 1]\n    while len(sequence) < n:\n        sequence.append(sequence[-1] + sequence[-2])\n    return sequence[:n]\n\n# Get the first 10 Fibonacci numbers\nfibonacci_numbers = fibonacci_sequence(10)\nfibonacci_numbers',
    'outputs': []},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_u20fmJV088azvbd5z0BUYUyz'},
 'type': 'message_creation'}

null


두 Step의 `step_details`를 볼 수 있음

1. `tool_calls` (하나의 Step에 하나 이상이 있을 수 있으므로 복수형)
2. `message_creation`

첫 번째 Step은 `tool_calls`로, 특히 `code_interpreter`를 사용하는데 여기에는 다음이 포함됨:

- 도구가 호출되기 전에 생성된 Python 코드인 `input`, 그리고
- Code Interpreter를 실행한 결과인 `output`.

두 번째 Step은 `message_creation`으로, 사용자에게 결과를 전달하기 위해 Thread에 추가된 `message`가 포함되어 있음

## <font color=yellow>File Search</font>

Assistants API의 또 다른 강력한 도구는 [file search](https://platform.openai.com/docs/assistants/tools/file-search/quickstart): 
- 질문에 답할 때 assistant가 지식 기반으로 사용할 파일을 업로드하는 기능 
- 이 기능도 대시보드 또는 API에서 활성화할 수 있으며, 사용하고자 하는 파일을 업로드할 수 있음
- `기존에 retrieve로 불렸는데, assistant API v2가 되면서 file search로 바뀜`
    - `OpenAI에서 제공하는 벡터DB RAG 인듯`

In [32]:
# Create a vector store
vector_store = client.vector_stores.create(name="machine learning")
 
# Ready the files for upload to OpenAI
file_paths = [r".\data\language_models_are_unsupervised_multitask_learners.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, 
    files=file_streams,
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [33]:
# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)
show_json(assistant)

{'id': 'asst_J8WwgAVAMAsKm3t5uPDLX5gA',
 'created_at': 1753188046,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'score_threshold': 0.0,
     'ranker': 'default_2024_08_21'}}}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': []},
  'file_search': {'vector_store_ids': ['vs_687f86f002548191af650550c16a5ede']}},
 'top_p': 1.0,
 'reasoning_effort': None}

In [34]:
thread, run = create_thread_and_run(
    "이 논문의 배경이 되는 수학 개념은 뭐야? 두 문장으로 설명해줘"
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\1054163418.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:14: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:17: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.create(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:23: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.messages.list(thread_id=thread.id, order="asc")


# Messages
user: 이 논문의 배경이 되는 수학 개념은 뭐야? 두 문장으로 설명해줘
assistant: 이 논문은 언어 모델의 수학적 개념으로 다중 작업 학습과 언어 모델링을 다루며, 다양한 작업을 학습하고 수행하기 위해 확률적 모델링을 사용합니다【4:0†source】.



> **Note**
> Retrieval에는 [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages)와 같은 더 복잡한 기능들이 있으며, 이는 다른 쿡북에서 다룰 수 있음


In [53]:
def get_mock_response_from_user_multiple_choice():
    return "a"

def get_mock_response_from_user_free_response():
    return "I don't know."

def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

다음은 샘플 퀴즈가 어떻게 보일지에 대한 예:

In [54]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "이름이 뭐야?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "가장 좋아하는 색이 뭐야?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["빨간색", "파랑색", "초록색", "노란색"],
        },
    ],
)
print("Responses:", responses)

Quiz: Sample Quiz

이름이 뭐야?

가장 좋아하는 색이 뭐야?
0. 빨간색
1. 파랑색
2. 초록색
3. 노란색

Responses: ["I don't know.", 'a']


이제, Assistant가 호출할 수 있도록 이 함수의 인터페이스를 JSON 형식으로 정의해 보겠음

In [55]:
function_json = {
    "name": "display_quiz",
    "description": "학생에게 퀴즈를 표시하고 학생의 응답을 반환합니다. 하나의 퀴즈에 여러 개의 질문이 포함될 수 있습니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "제목과 옵션(객관식인 경우)이 있는 질문의 배열입니다.",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

다시 한번, 대시보드나 API를 통해 우리의 Assistant를 업데이트해 보겠음

In [56]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "file_search"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_J8WwgAVAMAsKm3t5uPDLX5gA',
 'created_at': 1753188046,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'score_threshold': 0.0,
     'ranker': 'default_2024_08_21'}}},
  {'function': {'name': 'display_quiz',
    'description': '학생에게 퀴즈를 표시하고 학생의 응답을 반환합니다. 하나의 퀴즈에 여러 개의 질문이 포함될 수 있습니다.',
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': '제목과 옵션(객관식인 경우)이 있는 질문의 배열입니다.',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']},
         'choices': {'type': 'array', 'items': {'type': 'string'}}},
        

퀴즈를 요청

In [57]:
thread, run = create_thread_and_run(
    "두 가지 질문으로 퀴즈를 만들어줘. 하나는 주관식, 하나는 객관식으로, 그런 다음 답변에 대한 피드백을 보내줘"
)
run = wait_on_run(run, thread)
run.status

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\1054163418.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:14: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:17: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.create(


'requires_action'

하지만 이제 Run의 `status`를 확인하면 `requires_action`이 표시됨

In [58]:
show_json(run)

{'id': 'run_jSLoWPnUvEYgi3BGOTcrUT6f',
 'assistant_id': 'asst_J8WwgAVAMAsKm3t5uPDLX5gA',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1753189260,
 'expires_at': 1753189860,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_2Fa1mR4KEUvX88dGkBfhHQua',
     'function': {'arguments': '{"title": "Math Quiz", "questions": [{"question_text": "파이의 값은 무엇입니까?", "question_type": "FREE_RESPONSE"}, {"question_text": "다음 중 삼각형의 내각의 합은?", "question_type": "MULTIPLE_CHOICE", "choices": ["180도", "90도", "360도", "270도"]}]}',
      'name': 'display_quiz'},
     'type': 'function'},
    {'id': 'call_iwAhTTpxGddeBM3uFgDGwJz1',
     'function': {'arguments': '{"title": "Feedback Quiz", "questions": 

`required_action` 필드는 도구가 우리에게 실행하고 그 결과를 Assistant에게 제출하도록 대기 중인 것을 나타냄 
- 구체적으로 `display_quiz` 함수
- 먼저 `name` 및 `arguments`를 구문 분석하는 것부터 시작해 보겠음

> **참고**
> 이 경우에는 하나의 도구 호출하지만, 실제로는 Assistant는 여러 도구를 호출하기로 선택할 수 있

In [59]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: display_quiz
Function Arguments:


{'title': 'Math Quiz',
 'questions': [{'question_text': '파이의 값은 무엇입니까?',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': '다음 중 삼각형의 내각의 합은?',
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['180도', '90도', '360도', '270도']}]}

이제 Assistant에서 제공한 인수를 사용하여 실제로 `display_quiz` 함수를 호출해 보겠음

In [60]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: Math Quiz

파이의 값은 무엇입니까?

다음 중 삼각형의 내각의 합은?
0. 180도
1. 90도
2. 360도
3. 270도

Responses: ["I don't know.", 'a']


(이 응답들은 이전에 모의로 만들었던 것임, 실제로는 이 함수 호출에서 입력을 다시 받아올 것임)

이제 응답을 얻었으므로, Assistant에게 다시 제출
- 이전에 분석한 `tool_call`에서 찾을 수 있는 `tool_call` ID가 필요함 
- 또한 응답들의 `list`를 `str`로 인코딩해야 함

In [61]:
tool_outputs = []

for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    print("Function Name:", name)
    print("Function Arguments:", arguments)

    if name == "display_quiz":
        responses = display_quiz(arguments["title"], arguments["questions"])
        print("Responses:", responses)

        output = json.dumps(responses)
    else:
        output = json.dumps(f"No handler implemented for function: {name}")

    tool_outputs.append({
        "tool_call_id": tool_call.id,
        "output": output,
    })

# 툴 호출 결과 모두 제출
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_outputs,
)

show_json(run)

Function Name: display_quiz
Function Arguments: {'title': 'Math Quiz', 'questions': [{'question_text': '파이의 값은 무엇입니까?', 'question_type': 'FREE_RESPONSE'}, {'question_text': '다음 중 삼각형의 내각의 합은?', 'question_type': 'MULTIPLE_CHOICE', 'choices': ['180도', '90도', '360도', '270도']}]}
Quiz: Math Quiz

파이의 값은 무엇입니까?

다음 중 삼각형의 내각의 합은?
0. 180도
1. 90도
2. 360도
3. 270도

Responses: ["I don't know.", 'a']
Function Name: display_quiz
Function Arguments: {'title': 'Feedback Quiz', 'questions': [{'question_text': '파이의 값은 무엇입니까?', 'question_type': 'FREE_RESPONSE'}, {'question_text': '다음 중 삼각형의 내각의 합은?', 'question_type': 'MULTIPLE_CHOICE', 'choices': ['180도', '90도', '360도', '270도']}]}
Quiz: Feedback Quiz

파이의 값은 무엇입니까?

다음 중 삼각형의 내각의 합은?
0. 180도
1. 90도
2. 360도
3. 270도

Responses: ["I don't know.", 'a']


C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\4229150356.py:24: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.submit_tool_outputs(


{'id': 'run_jSLoWPnUvEYgi3BGOTcrUT6f',
 'assistant_id': 'asst_J8WwgAVAMAsKm3t5uPDLX5gA',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1753189260,
 'expires_at': 1753189860,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1753189261,
 'status': 'queued',
 'thread_id': 'thread_ARgYVUS6Q7zJpvoy1jUor4dr',
 'tool_choice': 'auto',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'score_threshold': 0.0,
     'ranker': 'default_2024_08_21'}}},
  {'function': {'name': 'display_quiz',
    'description': '학생에게 퀴즈를 표시하고 학생의 응답을 반환합니다. 하나의 퀴즈에 여러 개의 질문이 포함될 수 있습니다.',
    'parameters': {'type': 'object',
   

이제 다시 Run이 완료될 때까지 기다릴 수 있으며, Thread를 확인해 보겠음

In [63]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\2754427285.py:14: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8868\3121078466.py:23: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.messages.list(thread_id=thread.id, order="asc")


# Messages
user: 두 가지 질문으로 퀴즈를 만들어줘. 하나는 주관식, 하나는 객관식으로, 그런 다음 답변에 대한 피드백을 보내줘
assistant: 퀴즈에 대한 피드백을 드리겠습니다:

1. **주관식** - "파이의 값은 무엇입니까?": 일반적으로 3.14159로 표현됩니다.
2. **객관식** - "다음 중 삼각형의 내각의 합은?": 정답은 "180도" 입니다.



## 결론
간략함을 유지하기 위해 다루지 않은 몇 가지 섹션도 있으므로 더 자세히 살펴볼 수 있는 몇 가지 리소스를 제공함

- [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages): 파일 인용문 구문 분석
- [Files](https://platform.openai.com/docs/api-reference/assistants/file-object): Thread 범위 vs Assistant 범위
- [Parallel Function Calls](https://platform.openai.com/docs/guides/function-calling/parallel-function-calling): 하나의 단계에서 여러 도구 호출
- Multi-Assistant Thread Runs: 여러 Assistant에서 온 메시지가 있는 단일 Thread